****Import module****

* THANKS 
1. REF: https://github.com/Konthee/superai-3-Image-NER
1. REF: https://colab.research.google.com/drive/1-wrZmCyP9BQNG75fbY1XT0GHfHOcZztI?usp=sharing

In [ ]:
!pip -q install transformers
!pip -q install datasets
!pip -q install simpletransformers
!pip -q install python-crfsuite
!pip -q install pytesseract
!pip -q install deskew
!sudo apt-get install  tesseract-ocr libtesseract-dev tesseract-ocr-tha

In [ ]:
import cv2
import PIL
from PIL import Image
from PIL import ImageDraw
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import time
import numpy as np
import seaborn as sns
import pandas as pd
import shutil

from tqdm import tqdm
from tqdm.notebook import trange, tqdm
import pytesseract
import re
import seaborn as sns
sns.set_theme()
from deskew import determine_skew

# OCR

Skew correction / deskew

In [ ]:
# Rotate the image around its center
def rotateImage(cvImage, angle: float):
    newImage = cvImage.copy()
    (h, w) = newImage.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    newImage = cv2.warpAffine(newImage, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return newImage

# Deskew image
def deskew(cvImage):
    angle = determine_skew(cvImage)
    return rotateImage(cvImage, angle)

In [ ]:
kernel_sharp = np.array([[0,-1,0],
                         [-1,5,-1],
                         [0,-1,0]],dtype=np.float32)

Create folder with cut images

In [ ]:
path_image_cut= 'image_cut'
path_image = '/content/images/images'
try: shutil.rmtree(path_image_cut)
except : pass
#Create Folder
os.mkdir(path_image_cut)


kernel_sharp = np.array([[0,-1,0],
                   [-1,5,-1],
                   [0,-1,0]],dtype=np.float32)
num = 0
list_ = []
for index in range(len(os.listdir(path_image))):
    img = cv2.imread(f'{path_image}/{index:05d}.jpg',cv2.IMREAD_GRAYSCALE)
    img = cv2.filter2D(img,-1,kernel_sharp)
    img = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)
    deskew_ = deskew(img)
    imgresize2 = deskew_.copy()
    gray_blur = cv2.GaussianBlur(deskew_,(5,5),0)
    thresh=cv2.threshold(gray_blur, 180, 255, cv2.THRESH_BINARY +cv2.THRESH_OTSU)[1]
    kernel=np.ones((6,15),np.uint8)
    closing = cv2.erode(deskew_,kernel,iterations = 2)
    result_img=closing.copy()

    contours,hierachy=cv2.findContours(result_img,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    # w,h= result_img.shape

    # print(index)
    contours = contours[:-1][::-1]
    # print(len(contours),index)
    if len(contours) != 14:
      list_ += [index]
      while len(contours) < 14:
        new = np.array(contours[-1],dtype=np.int32)
        contours += (new,)


    print(len(contours),index)
    num = 14 *index
    for i, cnt in enumerate(contours):
        (x,y,w,h)= cv2.boundingRect(cnt)
        result = imgresize2[y:y+h,x:x+w]
        h_,w_ = result.shape


        result = cv2.fastNlMeansDenoising(result, None, 20, 7, 21)
        result = cv2.resize(result, (w_*8,h_*8), interpolation = cv2.INTER_AREA)
        result = cv2.GaussianBlur(result,(9 , 9),0)
        result = cv2.filter2D(result,-1,kernel_sharp)
        result=cv2.threshold(result,180,255,cv2.THRESH_OTSU)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
        result = cv2.dilate(result,kernel,iterations = 3)


        cv2.imwrite(path_image_cut+'/{:05d}.jpg'.format(num),result)
        num += 1




In [ ]:
import pytesseract
print(pytesseract.get_tesseract_version())
print(pytesseract.get_languages())

In [ ]:
def clean_data(txt,j):
    txt=txt.replace('\n','')
    txt=txt.replace('|','')

    if ","  in txt:
        txt=txt[txt.index(',')+1:]
    elif '.' in txt:
        txt=txt[txt.index('.')+1:]
    elif ' ' in txt :
        txt=txt[txt.index(' ')+1:]
    elif ':' in txt :
        txt=txt[txt.index(':')+1:]

    elif str(j) in txt :
        txt=txt[txt.index(str(j))+1:]

    txt=txt.replace(' ','')
    txt=txt.replace('-','')
    txt=txt.replace('๕','&')

    text=re.findall("[ก-๙]+", txt)
    num=re.findall("[0-9]+", txt)
    if (len(text) == len(num) != 0)  :
        txt=text[0]
    elif len(text) ==0 and len(num) >0 :
        if str(j) not in txt :
            txt=num[0]
        else : txt= '_'

    if txt == '':
        txt ='_'
    return txt

In [ ]:
path_image_cut = '/content/image_cut4'
df_test = pd.DataFrame({"Id" : [], "Text" : []})

add word in dataframe

In [ ]:
n=0
n+=1
import cv2
path_image_cut = '/content/image_cut4'
for j in range(len(os.listdir(path_image_cut))):
    local = path_image_cut + r'/{:05d}.jpg'.format(j)
    img = Image.open(local)
    txt = pytesseract.image_to_string(local, lang='tha',config='--psm 8') # --psm 13 --psm 8
    txt= clean_data(txt,j)
    df_test=df_test.append({"Id" : i, "Text" : txt}, ignore_index=True)

In [ ]:
df_new = df_test.copy()
df_new = df_new.reset_index()
df_new = df_new.drop('Id',axis = 1)
df_new = df_new.rename(columns={'index':'Id'})

# Tags NER

Train model

In [ ]:
import pandas as pd
import json
from datasets import load_dataset
PYTORCH_NO_CUDA_MEMORY_CACHING=1

lst20 = load_dataset("lst20", data_dir="/home/sivakorn/NER/AIFORTHAI/LST20_Corpus")
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
test_df = pd.DataFrame(lst20['test'])


#Column Filter only NER Tags
df_filter = ['id', 'tokens', 'ner_tags']
train_df = train_df[df_filter]
validation_df = validation_df[df_filter]
test_df = test_df[df_filter]


# Define Ner Tags replace missing ne_list.txt
NER_TAGS = [
       "O",
        "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "B_ORG",        "B_PER",        "B_TRM",        "B_TTL",
       "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
        "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "E_ORG",        "E_PER",        "E_TRM",        "E_TTL"]

def convert_data_to_df(df):
  data_df = pd.DataFrame()
  sentence_id = []
  words = []
  labels = []

  for sentence in range(len(df)):
    for token in range(len(df['tokens'][sentence])):
      sentence_id.append(sentence)
      words.append(df['tokens'][sentence][token])
      labels.append(NER_TAGS[df['ner_tags'][sentence][token]]) #Map 0 to "O", 1 to "B_BRN"

  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )

train_data = convert_data_to_df(train_df)
#Re-process to validate and test dataset
eval_data = convert_data_to_df(validation_df )
test_data = convert_data_to_df(test_df)
#train_data.head(9)

import logging
from simpletransformers.ner import NERModel, NERArgs
import torch


#torch.cuda.empty_cache()

# Simple Transformer https://simpletransformers.ai/docs/ner-minimal-start/
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)
max_seq_length = train_data['words'].str.len().max()

ner_args = NERArgs()
ner_args.train_batch_size = 128 #192 is fit for GPU T4, 512 for A100
ner_args.use_multiprocessing = True
ner_args.evaluate_during_training = True
ner_args.eval_batch_size = 128
ner_args.num_train_epochs = 12
ner_args.overwrite_output_dir = True
ner_args.gradient_accumulation_steps = 4
ner_args.learning_rate = 1e-5

model = NERModel(
     "camembert", # Model Type
     "airesearch/wangchanberta-base-att-spm-uncased",  #Ner Pre-trained Model
     args=ner_args, use_cuda=torch.cuda.is_available(), labels=NER_TAGS,ignore_mismatched_sizes=True) # Local Config


model.train_model(train_data, eval_data=eval_data)
result, model_outputs, preds_list = model.eval_model(eval_data)
print(result)

Prediction

In [ ]:
import logging
from simpletransformers.ner import NERModel, NERArgs
import torch
import pandas as pd 
texts_test = pd.read_csv('/home/sivakorn/OCR_NER/df_final.csv')
#print(texts_test.columns)
texts_test['Text'] = texts_test['Text'].replace({ r'\x0c': '', r'\n': ''}, regex=True)
NER_TAGS = [
       "O",
        "B_BRN",        "B_DES",        "B_DTM",        "B_LOC",        "B_MEA",        "B_NUM",        "B_ORG",        "B_PER",        "B_TRM",        "B_TTL",
       "I_BRN",        "I_DES",        "I_DTM",        "I_LOC",        "I_MEA",        "I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
        "E_BRN",        "E_DES",        "E_DTM",        "E_LOC",        "E_MEA",        "E_NUM",        "E_ORG",        "E_PER",        "E_TRM",        "E_TTL"]


texts_test_raw = texts_test['Text'].tolist()
def blank_space(x):
  if x == '':
    x = '_'
  return x

#Loop replace blank to "_"
for i in range(len(texts_test_raw)):
  texts_test_raw[i] = blank_space(texts_test_raw[i])
print(len(texts_test_raw))

def data_inside(data_list):
  x = 0
  for i in range(len(data_list)):
    a = len(data_list[i])
    x = x+a
  return x

def split_into_sentences(tokens, tokens_per_sentence=20):
    sentences = []
    for i in range(0, len(tokens), tokens_per_sentence):
        sentence = tokens[i:i+tokens_per_sentence]
        sentences.append(sentence)
    return sentences


my_token = split_into_sentences(texts_test_raw)
print(my_token[0:20])
print(len(my_token))
my_token_list = []

for i in range(len(my_token)):
  sent_join = ' '.join(str(my_token[i]))
  #print(sent_join)
  my_token_list.append(sent_join)


print(len(my_token))
print(len(my_token_list))
print(data_inside(my_token_list))


ner_args = NERArgs()
ner_args.eval_batch_size = 128
ner_args.use_multiprocessing = True
#ner_args.max_seq_length = 500 # Fixed Requirement ##############################
model = NERModel(
     "auto", "/home/sivakorn/OCR_NER/AIFORTHAI/outputs/best_model/", args=ner_args, use_cuda=torch.cuda.is_available(), labels= NER_TAGS  # your latest model
)

predictions, raw_outputs = model.predict(my_token, False)
print(predictions[0][0:6])
print(data_inside(predictions))

final_test_df = []
for i in range(len(predictions)):
# for i in range(3):
  for j in range(len(predictions[i])):
    data = predictions[i][j]
    value = data.values()
    final_test_df += value

final_result = pd.DataFrame(final_test_df)
data = {
    'New_id': [
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
        10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30, 31
    ],
    'New_tag': [
        'O', 'B_ORG', 'B_PER', 'B_LOC', 'B_MEA', 'I_DTM', 'I_ORG', 'E_ORG', 'I_PER', 'B_TTL',
        'E_PER', 'B_DES', 'E_LOC', 'B_DTM', 'B_NUM', 'I_MEA', 'E_DTM', 'E_MEA', 'I_LOC', 'I_DES',
        'E_DES', 'I_NUM', 'E_NUM', 'B_TRM', 'B_BRN', 'I_TRM', 'E_TRM', 'I_TTL', 'I_BRN', 'E_BRN',
        'E_TTL', 'B_NAME'
    ]
}
# Create DataFrame
tag_df = pd.DataFrame(data)
final_result['New_id'] = final_result[0].map(tag_df.set_index('New_tag')['New_id'])
submisstion_df = pd.read_csv("/home/sivakorn/OCR_NER/sample_submission.csv")
submisstion_df['token'] = pd.DataFrame({'Token': texts_test_raw})
submisstion_df['Final_pred'] = pd.DataFrame({'Final_pred': final_test_df})
submisstion_df['Predicted'] = final_result['New_id']
submission_df = submisstion_df[['Id','Predicted']]
submission_df = submission_df.rename(columns={'Predicted': 'pred'})
submission_df = submission_df.set_index('Id')

submission_df.to_csv("/home/sivakorn/OCR_NER/ocr_ner3.csv")